# Project Zeus
## Created by Michael Wentz

### Import Modules

In [32]:
import numpy as np
import pandas as pd
import math
from ipywidgets import interact, interactive, fixed, interact_manual, VBox, FloatSlider
import ipywidgets as widgets

### Load production data from excel file

In [33]:
file = 'https://raw.githubusercontent.com/mwentzWW/petrolpy/master/Zeus/Sample_Production/Sample_Prod_Data.xlsx'
well_data = pd.read_excel(file)
well_data.columns = [c.lower() for c in well_data.columns]
well_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 5 columns):
api number         704 non-null object
production date    704 non-null datetime64[ns]
oil_bbl            694 non-null float64
gas_mcf            694 non-null float64
water_bbl          693 non-null float64
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 27.6+ KB


### Check Data Layout

In [34]:
well_data.head()

,api number,production date,oil_bbl,gas_mcf,water_bbl
0,40 063 05037,1959-05-31,3533.0,0.0,0.0
1,40 063 05037,1959-06-30,4763.0,0.0,0.0
2,40 063 05037,1959-07-31,4475.0,1570.0,0.0
3,40 063 05037,1959-08-31,2620.0,917.0,0.0
4,40 063 05037,1959-09-30,3531.0,1236.0,0.0


### Delete API because there is only one well, and we don't need water

In [35]:
well_data.drop(columns=['api number', 'water_bbl'], inplace=True)
well_data.head()

,production date,oil_bbl,gas_mcf
0,1959-05-31,3533.0,0.0
1,1959-06-30,4763.0,0.0
2,1959-07-31,4475.0,1570.0
3,1959-08-31,2620.0,917.0
4,1959-09-30,3531.0,1236.0


### Replace NaN with zeros for computations

In [36]:
well_data.fillna(value=0, inplace=True)

### Calculate effective decline rate for oil and gas

In [37]:
well_data.oil_bbl = well_data.oil_bbl/30
well_data.gas_mcf = well_data.gas_mcf/30

qi_oil = well_data.oil_bbl[0:30].max()
qi_gas = well_data.gas_mcf[0:30].max()

In [38]:
well_data['eff_decline_oil'] = -(well_data.oil_bbl - qi_oil)/qi_oil
well_data['eff_decline_gas'] = -(well_data.gas_mcf - qi_gas)/qi_gas

In [39]:
well_data.head()

,production date,oil_bbl,gas_mcf,eff_decline_oil,eff_decline_gas
0,1959-05-31,117.766667,0.000000,0.258241,1.000000
1,1959-06-30,158.766667,0.000000,-0.000000,1.000000
2,1959-07-31,149.166667,52.333333,0.060466,-0.000000
3,1959-08-31,87.333333,30.566667,0.449927,0.415924
4,1959-09-30,117.700000,41.200000,0.258661,0.212739


### Normalize data by time

In [58]:
num_days_online = len(well_data['oil_bbl'])
num_days_online

704

In [59]:
days = []
for day in range(0, num_days_online):
    days.append(day)
days[0:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [60]:
well_data['time'] = days
well_data.head()

,production date,oil_bbl,gas_mcf,eff_decline_oil,eff_decline_gas,time,type_curve_oil
0,1959-05-31 00:00:00,117.766667,0.000000,0.258241,1.000000,0,158.766667
1,1959-06-30 00:00:00,158.766667,0.000000,-0.000000,1.000000,1,137.385974
2,1959-07-31 00:00:00,149.166667,52.333333,0.060466,-0.000000,2,120.050410
3,1959-08-31 00:00:00,87.333333,30.566667,0.449927,0.415924,3,105.800361
4,1959-09-30 00:00:00,117.700000,41.200000,0.258661,0.212739,4,93.944773


### Calculate the type curve for oil and gas, assume b = 0.5 (will adjust later), use first 30 day average decline for di

In [61]:
well_data.fillna(value=0, inplace=True)
well_data.loc[well_data.eff_decline_oil > 10000] = 0
well_data.loc[well_data.eff_decline_gas > 10000] = 0
well_data.time.astype(float)

0        0.0
1        1.0
2        2.0
3        3.0
4        4.0
5        5.0
6        6.0
7        7.0
8        8.0
9        9.0
10      10.0
11      11.0
12      12.0
13      13.0
14      14.0
15      15.0
16      16.0
17      17.0
18      18.0
19      19.0
20      20.0
21      21.0
22      22.0
23      23.0
24      24.0
25      25.0
26      26.0
27      27.0
28      28.0
29      29.0
       ...  
674    674.0
675    675.0
676    676.0
677    677.0
678    678.0
679    679.0
680    680.0
681    681.0
682    682.0
683    683.0
684    684.0
685    685.0
686    686.0
687    687.0
688    688.0
689    689.0
690    690.0
691    691.0
692    692.0
693    693.0
694    694.0
695    695.0
696    696.0
697    697.0
698    698.0
699    699.0
700    700.0
701    701.0
702    702.0
703    703.0
Name: time, Length: 704, dtype: float64

In [62]:
print(well_data.head())


       production date     oil_bbl    gas_mcf  eff_decline_oil  \
0  1959-05-31 00:00:00  117.766667   0.000000         0.258241   
1  1959-06-30 00:00:00  158.766667   0.000000        -0.000000   
2  1959-07-31 00:00:00  149.166667  52.333333         0.060466   
3  1959-08-31 00:00:00   87.333333  30.566667         0.449927   
4  1959-09-30 00:00:00  117.700000  41.200000         0.258661   

   eff_decline_gas  time  type_curve_oil  
0         1.000000     0      158.766667  
1         1.000000     1      137.385974  
2        -0.000000     2      120.050410  
3         0.415924     3      105.800361  
4         0.212739     4       93.944773  


In [63]:
print(well_data.loc[[3], ('gas_mcf')])

3    30.566667
Name: gas_mcf, dtype: float64


In [70]:
def plot_oil(b_value=0.5, initial_decline_rate=0.15, start_of_exp=well_data.time.max(), exp_yes_no=False):
    well_data['type_curve_oil'] = qi_oil*((1 + b_value*initial_decline_rate*well_data.time)**(-1/b_value))
    if exp_yes_no == True:
        start_of_exp = int(start_of_exp)
        well_data['type_curve_expon'] = well_data.loc[start_of_exp, ('type_curve_oil')]*2.71828**(-initial_decline_rate*(well_data.time-start_of_exp))
        well_data.loc[start_of_exp:, ('type_curve_oil')] = 0
        well_data.loc[0:start_of_exp-1, ('type_curve_expon')] = 0
        connector = well_data.loc[start_of_exp, ('type_curve_expon')]
        well_data.type_curve_oil = well_data.type_curve_oil + well_data.type_curve_expon
        well_data.type_curve_expon.astype('float')
    well_data.type_curve_oil.astype('float')
    well_data.fillna(value=0, inplace=True)
    well_data.loc[well_data.type_curve_oil > qi_oil] = qi_oil

    fig, axes = plt.subplots( figsize=(12, 6))

    axes.plot(well_data.time, well_data.type_curve_oil, 'k--', lw =2, label='Type Curve')
    axes.plot(well_data.time, well_data.oil_bbl, 'r', label='Historical Production')
    if exp_yes_no == True:    
        axes.plot(start_of_exp, connector, 'bx', label='Exponential Decline')
    axes.legend(loc=1);
    axes.grid(True)
    axes.set_xlabel('Time (Months)')
    axes.set_ylabel('Oil Rate (BPD)')
    axes.set_title('Oil Type Curve');

def plot_gas(b_value=0.5, initial_decline_rate=0.15):
    well_data['type_curve_gas'] = qi_gas*((1 + b_value*initial_decline_rate*well_data.time)**(-1/b_value))
    well_data.type_curve_gas.astype('int')
    well_data.fillna(value=0, inplace=True)
    well_data.loc[well_data.type_curve_gas > qi_gas] = qi_gas
    
    fig, axes = plt.subplots(figsize=(12, 6))

    axes.plot(well_data.time, well_data.type_curve_gas, 'k--', lw=2, label='Type Curve')
    axes.plot(well_data.time, well_data.gas_mcf, 'g', label='Historical Production')
    axes.legend(loc=1);
    axes.grid(True)
    axes.set_xlabel('Time (Months)')
    axes.set_ylabel('Gas Rate (MCFD)')
    axes.set_title('Oil Type Curve');

### Plot production with type curves

In [71]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [72]:
interact_oil = interact(plot_oil, b_value= (0, 1, 0.01), initial_decline_rate= (0, 1, 0.01), start_of_exp= (1, well_data.time.max(), 1), exp_yes_no = False)

interactive(children=(FloatSlider(value=0.5, description='b_value', max=1.0, step=0.01), FloatSlider(value=0.1…

In [49]:
well_data

,production date,oil_bbl,gas_mcf,eff_decline_oil,eff_decline_gas,time,type_curve_oil
0,1959-05-31 00:00:00,117.766667,0.000000,0.258241,1.000000,0.0,158.766667
1,1959-06-30 00:00:00,158.766667,0.000000,-0.000000,1.000000,1.0,137.385974
2,1959-07-31 00:00:00,149.166667,52.333333,0.060466,-0.000000,2.0,120.050410
3,1959-08-31 00:00:00,87.333333,30.566667,0.449927,0.415924,3.0,105.800361
4,1959-09-30 00:00:00,117.700000,41.200000,0.258661,0.212739,4.0,93.944773
5,1959-10-31 00:00:00,106.533333,37.300000,0.328994,0.287261,5.0,83.975758
6,1959-11-30 00:00:00,105.433333,36.900000,0.335923,0.294904,6.0,75.513278
7,1959-12-31 00:00:00,66.566667,23.333333,0.580726,0.554140,7.0,68.268387
8,1960-01-31 00:00:00,90.966667,0.000000,0.427042,1.000000,8.0,62.018229
9,1960-02-29 00:00:00,89.333333,0.000000,0.437329,1.000000,9.0,56.588698


## To-do

* add multiline segment functionality
* add optimized solution by minimizing the error